##Задача 1

In [108]:
def get_stat(mas):
  mean = sum(mas)/len(mas)  # mean is finded
  d = dict()
  for elem in mas:
    d[elem] = d.get(elem, 0) + 1
  to_sort = []
  for k, v in d.items():
    to_sort.append((v, k))   #creating a list of tuples like (count, elem) and then sort it to get max count
  to_sort.sort()
  mode = to_sort[-1][1]
  mas.sort()
  if len(mas) % 2:                  # 2 variants of counting median
    median = mas[int(len(mas) / 2)]
  else:
    median = (mas[int(len(mas) / 2) - 1] + mas[int(len(mas) / 2)]) / 2
  return {"mean" : mean, "median" : median, "mode" : mode}

Протестируем get_stat

In [109]:
mas = [4, 2, 3, 1, 100, 4, 100, 2]
print(get_stat(mas))

{'mean': 27.0, 'median': 3.5, 'mode': 100}


In [110]:
mas2 = [5, 7, 10, 8, 7]
print(get_stat(mas2))

{'mean': 7.4, 'median': 7, 'mode': 7}


##Задача 2

In [111]:
def parse_csv(file_path):
  with open(file_path, 'r') as f:
    titles = f.readline()             # getting the first line
    titles = titles.replace('\n', '') # just perfectionism
    titles = titles.split(',')
    titles[0] = 'index'
    strings = []
    while True:
      line = f.readline()           #getting the other lines
      if not line:
        break
      else:
        line = line.replace('\n', '')  #just perfectionism as well
        line = line.replace("'", '')
        line = line.replace('"', '')
        line = line.split(',', 7)
        line[-1] = line[-1].replace('[', '')
        line[-1] = line[-1].replace(']', '')
        line[-1] = line[-1].split(', ')    # then, roles looks like list of str
        strings.append(line)
    parsed = []
    for line in strings:
      d = {}
      for i, col in enumerate(titles):
        d[col] = int(line[i]) if isinstance(line[i], str) and line[i].isnumeric() else line[i]    # if we can get a number instead of string, we do it
      parsed.append(d)
  return parsed

Спарсим нужный файл, отсортируем результат по параметру legs и выведем героя с максимумом

In [112]:
pars = parse_csv('/content/dota_hero_stats.csv')      # I use colab, so replace the beginning of the path if it needed
pars_srt = sorted(pars, key = lambda x: x['legs'], reverse = True)
pars_srt[0]

{'index': 59,
 'attack_type': 'Melee',
 'id': 61,
 'legs': 8,
 'localized_name': 'Broodmother',
 'name': 'npc_dota_hero_broodmother',
 'primary_attr': 'agi',
 'roles': ['Carry', 'Pusher', 'Escape', 'Nuker']}

##Мера близости

Логично было бы использовать меру близости как равенство переменных attack_type, legs, primary_attr и количество совпадений элементов из roles. Например, если у одного персонажа roles = ['Support', 'Disabler', 'Nuker', 'Durable'] \\
То из 2 вариантов: \\
- ['Initiator', 'Durable', 'Disabler', 'Jungler'] \\
- ['Carry', 'Escape', 'Nuker']

больше подходит вариант 1, поскольку там 2 совпадения, а со вторым вариантом всего одно.

In [113]:
def get_sim(hero1, hero2):
  cnt = 0
  if hero1['attack_type'] == hero2['attack_type'] and hero1['legs'] == hero2['legs'] and hero1['primary_attr'] == hero2['primary_attr']:
    roles1 = sorted(hero1['roles'])
    roles2 = sorted(hero2['roles'])
    i, j = 0, 0
    while i != len(roles1) and j != len(roles2):
      if roles1[i] == roles2[j]:
        cnt += 1
        i += 1                #use method of 2 pointers
        j += 1
      elif roles1[i] > roles2[j]:
        j += 1
      else:
        i += 1
  return cnt

In [114]:
mas_of_sims = []
for i in range(len(pars) - 1):
  for j in range(i + 1, len(pars)):
    mas_of_sims.append((get_sim(pars[i], pars[j]), i, j, pars[i]['localized_name'], pars[j]['localized_name']))
mas_of_sims.sort(reverse = True)
max_s = mas_of_sims[0][0]
for elem in mas_of_sims:
  if elem[0] == max_s:      # print all pairs that has max
    print(elem)
  else:
    break

(6, 22, 71, 'Kunkka', 'Alchemist')
(6, 18, 47, 'Tiny', 'Dragon Knight')
